In [1]:
import time
import os 

start_time = time.time() # measure run time

WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')


import yfinance as yf
# import pandas as pd
import numpy as np

from utils.finance_utils import *
from utils.helpers import separate_corr_pairs

from utils.config import PROGRAM_START_DATE, PROGRAM_END_DATE, TOP_N_STOCKS

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py

---------------------------------
finance_utils.py successfully loaded, updated last Feb. 04 2025 1:01
---------------------------------



---------------------------------
helpers.py successfully loaded, updated last Feb. 04 2025
---------------------------------


Updated on 02/04/2025 1:46


Sector Valuation: Group by sectors, and find best stock in each sector

In [2]:
filtered_valuation_df = pd.read_csv(f'{WORKSPACE_DIR}/portfolio_py/data/clean/filtered_valuation_df.csv')
# Add sector column to the DataFrame
filtered_valuation_df['sector'] = filtered_valuation_df['Tickers'].apply(get_sector)
filtered_valuation_df.head()

,marketCap,trailingPE,profitMargins,trailingEps,Tickers,Sharpe_ratios,sector
0,2.905739e+12,46.897236,0.55041,2.53,NVDA,3.295244,Technology
1,1.414962e+11,53.741630,0.40294,2.09,ANET,2.474921,Technology
2,5.361997e+10,12.412737,0.39410,4.24,MPLX,2.439091,Energy
3,3.065550e+12,33.228848,0.35608,12.41,MSFT,2.195851,Technology
4,1.784178e+12,29.525785,0.35550,23.85,META,3.291678,Communication Services


In [3]:
# possibly change group by  
print('\n----Filtering Df----')


top_stocks_on_profit = get_top_n_by_sector(filtered_valuation_df, filter_var='profitMargins', top_n=3)
top_stocks_on_sharpe = get_top_n_by_sector(filtered_valuation_df, filter_var='Sharpe_ratios', top_n=3)

print(f'Filtered based on: Profit Margins')
print(f"Sector Count: {len(top_stocks_on_profit['sector'].unique())}")
print(f"We have the following sector(s): \n{top_stocks_on_profit['sector'].unique()}")

print(f'\nFiltered based on: Sharpe Ratio')
print(f"Sector Count: {len(top_stocks_on_sharpe['sector'].unique())}")
print(f"We have the following sector(s): \n{top_stocks_on_sharpe['sector'].unique()}")


----Filtering Df----
Filtered based on: Profit Margins
Sector Count: 8
We have the following sector(s): 
['Communication Services' 'Consumer Cyclical' 'Energy'
 'Financial Services' 'Healthcare' 'Industrials' 'Technology' 'Utilities']

Filtered based on: Sharpe Ratio
Sector Count: 8
We have the following sector(s): 
['Communication Services' 'Consumer Cyclical' 'Energy'
 'Financial Services' 'Healthcare' 'Industrials' 'Technology' 'Utilities']


Diversifaction: Find negative correlated portfolios
    Gotta get histoical price, and do it on that

In [4]:
print('\n----Begining Diversifaction----')
profit_tickers_to_do_corr = top_stocks_on_profit['Tickers'].tolist()
top_profit_corr_pairs = get_corr_pairs_of_stocks(profit_tickers_to_do_corr)

sharpe_tickers_to_do_corr = top_stocks_on_sharpe['Tickers'].tolist()
top_sharpe_corr_pairs = get_corr_pairs_of_stocks(sharpe_tickers_to_do_corr)


[******************    38%                       ]  6 of 16 completed


----Begining Diversifaction----


[*********************100%***********************]  16 of 16 completed
[*********************100%***********************]  16 of 16 completed


Get Correlation, but need to find a way to find optimal pair, such that corr is low, all around portofolio

In [5]:
corr_profit_pairs = top_profit_corr_pairs.index.to_list()
corr_sharpe_pairs = top_sharpe_corr_pairs.index.to_list()

In [6]:
mpt_stocks_for_sharpe = separate_corr_pairs(corr_sharpe_pairs, top_n=TOP_N_STOCKS)
mpt_stocks_for_profit = separate_corr_pairs(corr_profit_pairs, top_n=TOP_N_STOCKS)
print(f'Top Stocks based on Sharpe: {mpt_stocks_for_sharpe}')
print(f'Top Stocks based on Profit Margin: {mpt_stocks_for_profit}')

Top Stocks based on Sharpe: ['APP' 'ITUB' 'BKNG' 'MPLX' 'PDD' 'HLT' 'NVDA' 'META']
Top Stocks based on Profit Margin: ['BKNG' 'ITUB' 'MPLX' 'ANET' 'HLT' 'PDD' 'NVDA' 'CPRT']


In [7]:
with open('utils/config.py', 'a') as f:
    f.write('\n# New Entry from portfolio_selection.ipynb')
    f.write(f'\nmpt_stocks_for_sharpe = {(mpt_stocks_for_sharpe).tolist()}\n')
    f.write(f'mpt_stocks_for_profit = {(mpt_stocks_for_profit).tolist()}\n')